In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

from utils.real_time_tools import RealTimeTools
from utils.dataset_tools import DatasetTools

In [ ]:
# Sprawdzenie czy widzi kartę graficzną
print(tf.config.list_physical_devices('GPU'))

In [2]:
model = load_model('../../model/2025-10-12-19:35:37/cnn1d_model.keras')

I0000 00:00:1760726764.034082  120202 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1731 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1


In [3]:
with open('../../model/2025-10-12-19:35:37/labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

In [4]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7
)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1760726808.135553  120424 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760726808.155179  120424 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


### Test rozpoznawania liter w czasie rzeczywistym

In [14]:
sequence = []
THRESHOLD =0.8
cap = cv2.VideoCapture(0)
try:
    if not cap.isOpened():
        print("Camera is not opened")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("No frames")
            break

        image_rbg = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rbg)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = RealTimeTools.extract_hand_landmarks(hand_landmarks)

                if landmarks is not None:
                    sequence.append(landmarks)

                    if len(sequence) >= DatasetTools.SEQUENCE_SIZE:
                        seq = sequence[-DatasetTools.SEQUENCE_SIZE:]
                        norm_sequence = DatasetTools.normalize_wrist(seq)
                        input_sequence = norm_sequence.reshape(1, DatasetTools.SEQUENCE_SIZE, DatasetTools.FEATURE_SIZE)

                        prediction = model.predict(input_sequence)[0]
                        max_prob = np.max(prediction)
                        label = label_names[np.argmax(prediction)]

                        if max_prob > THRESHOLD:
                            cv2.putText(frame,f'{label} ({max_prob:.2f})',(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
                            sequence.pop(0)
                else:
                    sequence = []

        cv2.imshow("Gesture Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Camera stopped")
finally:
    cap.release()
    cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

### Test pisania w czasie rzeczywistym